# Benchmaring metrics 

## Step 0- Install packages 

The `pbxplore` package provides tools for analyzing and visualizing Protein Block sequences, offering advanced methods for protein structure analysis. `pooch` is a Python package that manages the download of data files and their local storage, making it easier to fetch and cache datasets needed for analysis. Both packages can be installed using pip: `pip install pbxplore pooch`.

In [1]:
! pip install pbxplore pooch

In [17]:
import time
import tarfile
import pbxplore as pbx
import numpy as np
import pooch
from menger.data import files
from menger.analysis.mengercurvature import MengerCurvature
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align


## Step 1- Data retrieval

We utilize the `pooch` package to fetch molecular dynamics (MD) simulation data from a Zenodo repository. This data includes both the topology (PSF) and trajectory (DCD) files of a minimized all-atom simulation of the Tau protein wild-type sequence. 


In [8]:


# Create a pooch object with the base URL and compressed file handling
zenodo = pooch.create(
    path="./.data",  # Local storage location
    base_url="https://zenodo.org/records/14900596/files/",
    registry={
        "Zenodo_Rh_pCALVADOS.tar": None  # The compressed archive
    }
)

# First, fetch the compressed archive
tar_file = zenodo.fetch("Zenodo_Rh_pCALVADOS.tar")

# Extract the specific files from the archive with correct paths
with tarfile.open(tar_file) as tar:
    tar.extract("Zenodo_Rh_pCALVADOS/Trajectory_sequence_WT/all_atom_minimized_trajectory/topology_Tau_minimized_all_atoms_sequence_WT_renumerote.psf", path="./.data")
    tar.extract("Zenodo_Rh_pCALVADOS/Trajectory_sequence_WT/all_atom_minimized_trajectory/trajectory_Tau_minimized_all_atoms_sequence_WT.dcd", path="./.data")

# Update registry with correct paths
zenodo.registry.update({
    "Zenodo_Rh_pCALVADOS/Trajectory_sequence_WT/all_atom_minimized_trajectory/topology_Tau_minimized_all_atoms_sequence_WT_renumerote.psf": None,
    "Zenodo_Rh_pCALVADOS/Trajectory_sequence_WT/all_atom_minimized_trajectory/trajectory_Tau_minimized_all_atoms_sequence_WT.dcd": None
})

# Fetch the files from the extracted location
topology_file = zenodo.fetch("Zenodo_Rh_pCALVADOS/Trajectory_sequence_WT/all_atom_minimized_trajectory/topology_Tau_minimized_all_atoms_sequence_WT_renumerote.psf")
trajectory_file = zenodo.fetch("Zenodo_Rh_pCALVADOS/Trajectory_sequence_WT/all_atom_minimized_trajectory/trajectory_Tau_minimized_all_atoms_sequence_WT.dcd")

print(f"PSF file downloaded to: {topology_file}")
print(f"DCD file downloaded to: {trajectory_file}")

PSF file downloaded to: /home/etienne-reboul/projects/Menger_curvature_Daskified/menger_curvature/notebooks/.data/Zenodo_Rh_pCALVADOS/Trajectory_sequence_WT/all_atom_minimized_trajectory/topology_Tau_minimized_all_atoms_sequence_WT_renumerote.psf
DCD file downloaded to: /home/etienne-reboul/projects/Menger_curvature_Daskified/menger_curvature/notebooks/.data/Zenodo_Rh_pCALVADOS/Trajectory_sequence_WT/all_atom_minimized_trajectory/trajectory_Tau_minimized_all_atoms_sequence_WT.dcd



## Step 2- Benchmark Neq
We will perform 3 replicas to benchmark the computation of the neq metric from PBxplore on both Tau protein and tubulin chain A molecular dynamics data.


In [4]:
def benchmark_pbxplore(trajectory_file, topology_file, n_replicas=3):
    """
    Benchmark PBxplore analysis performance.
    
    Parameters:
    -----------
    dcd_file : str
        Path to the DCD trajectory file
    psf_file : str
        Path to the PSF topology file
    n_replicas : int, optional
        Number of benchmark replicas to run (default=3)
        
    Returns:
    --------
    tuple
        (mean_time, std_time) in seconds
    """
    times = []
    
    for _ in range(n_replicas):
        start_time = time.perf_counter()
        
        # Original analysis code
        sequences = []
        for chain_name, chain in pbx.chains_from_trajectory(trajectory_file, topology_file):
            dihedrals = chain.get_phi_psi_angles()
            pb_seq = pbx.assign(dihedrals)
            sequences.append(pb_seq)

        count_matrix = pbx.analysis.count_matrix(sequences)
        neq_by_position = pbx.analysis.compute_neq(count_matrix)
        
        end_time = time.perf_counter()
        execution_time = end_time - start_time
        times.append(execution_time)

    mean_time = np.mean(times)
    std_time = np.std(times)
    
    return mean_time, std_time

In [5]:
files.TUBULIN_CHAIN_A_DCD

PosixPath('/home/etienne-reboul/projects/Menger_curvature_Daskified/menger_curvature/menger/data/test_data/tubulin_chain_a.dcd')

In [ ]:
# benchmark on Tau protein MD
mean_time, std_time = benchmark_pbxplore(trajectory_file,topology_file , n_replicas=3)

print(f"Mean execution time: {mean_time:.2f} ± {std_time:.2f} seconds")

Frame 1/20000.
Frame 100/20000.
Frame 200/20000.
Frame 300/20000.
Frame 400/20000.
Frame 500/20000.
Frame 600/20000.
Frame 700/20000.
Frame 800/20000.
Frame 900/20000.
Frame 1000/20000.
Frame 1100/20000.
Frame 1200/20000.
Frame 1300/20000.
Frame 1400/20000.
Frame 1500/20000.
Frame 1600/20000.
Frame 1700/20000.
Frame 1800/20000.
Frame 1900/20000.
Frame 2000/20000.
Frame 2100/20000.
Frame 2200/20000.
Frame 2300/20000.
Frame 2400/20000.
Frame 2500/20000.
Frame 2600/20000.
Frame 2700/20000.
Frame 2800/20000.
Frame 2900/20000.
Frame 3000/20000.
Frame 3100/20000.
Frame 3200/20000.
Frame 3300/20000.
Frame 3400/20000.
Frame 3500/20000.
Frame 3600/20000.
Frame 3700/20000.
Frame 3800/20000.
Frame 3900/20000.
Frame 4000/20000.
Frame 4100/20000.
Frame 4200/20000.
Frame 4300/20000.
Frame 4400/20000.
Frame 4500/20000.
Frame 4600/20000.
Frame 4700/20000.
Frame 4800/20000.
Frame 4900/20000.
Frame 5000/20000.
Frame 5100/20000.
Frame 5200/20000.
Frame 5300/20000.
Frame 5400/20000.
Frame 5500/20000.
Fram

Mean execution time: 218.48 ± 1.17 seconds


In [ ]:
# Benchmark the analysis on the tubulin data
mean_time, std_time = benchmark_pbxplore(files.TUBULIN_CHAIN_A_DCD, files.TUBULIN_CHAIN_A_PDB, n_replicas=3)

print(f"Mean execution time: {mean_time:.2f} ± {std_time:.2f} seconds")

Frame 1/1000.
Frame 100/1000.
Frame 200/1000.
Frame 300/1000.
Frame 400/1000.
Frame 500/1000.
Frame 600/1000.
Frame 700/1000.
Frame 800/1000.
Frame 900/1000.
Frame 1000/1000.
Frame 1000/1000.
Frame 1/1000.
Frame 100/1000.
Frame 200/1000.
Frame 300/1000.
Frame 400/1000.
Frame 500/1000.
Frame 600/1000.
Frame 700/1000.
Frame 800/1000.
Frame 900/1000.
Frame 1000/1000.
Frame 1000/1000.
Frame 1/1000.
Frame 100/1000.
Frame 200/1000.
Frame 300/1000.
Frame 400/1000.
Frame 500/1000.
Frame 600/1000.
Frame 700/1000.
Frame 800/1000.
Frame 900/1000.


Mean execution time: 10.54 ± 0.17 seconds


Frame 1000/1000.
Frame 1000/1000.


## Step 3- Benchmark RMSF
We will perform 3 replicas to benchmark the computation of the Root Mean Square Fluctuation (RMSF) from MDAnalysis implementation on both Tau protein and tubulin chain A molecular dynamics data.

In [14]:
def benchmark_rmsf(topology_file, trajectory_file, n_replicas=3):
    """
    Benchmark RMSF analysis performance using MDAnalysis.
    
    Parameters:
    -----------
    topology_file : str
        Path to the topology file (PSF/PDB)
    trajectory_file : str
        Path to the trajectory file (DCD)
    n_replicas : int, optional
        Number of benchmark replicas to run (default=3)
        
    Returns:
    --------
    tuple
        (mean_time, std_time) in seconds
    """
    times = []
    
    for _ in range(n_replicas):
        start_time = time.perf_counter()
        
        # Load the system
        u = mda.Universe(topology_file, trajectory_file)
        
        # Calculate average structure and align
        average = align.AverageStructure(u, u, select='protein and name CA',
                                       ref_frame=0).run()
        ref = average.results.universe
        aligner = align.AlignTraj(u, ref,
                                select='protein and name CA',
                                in_memory=True).run()
        
        # Calculate RMSF
        c_alphas = u.select_atoms('protein and name CA')
        R = rms.RMSF(c_alphas).run()
        
        end_time = time.perf_counter()
        execution_time = end_time - start_time
        times.append(execution_time)

    mean_time = np.mean(times)
    std_time = np.std(times)
    
    return mean_time, std_time

In [15]:
# Benchmark RMSF on Tau protein MD
mean_time, std_time = benchmark_rmsf(topology_file, trajectory_file, n_replicas=3)

print(f"Mean execution time: {mean_time:.2f} ± {std_time:.2f} seconds")

Mean execution time: 22.96 ± 0.44 seconds


In [16]:
# Benchmark RMSF on Tubulin chain A MD
mean_time, std_time = benchmark_rmsf(files.TUBULIN_CHAIN_A_PDB, files.TUBULIN_CHAIN_A_DCD, n_replicas=3)

print(f"Mean execution time: {mean_time:.2f} ± {std_time:.2f} seconds")

Mean execution time: 1.21 ± 0.02 seconds


## Step 4- Benchmark Menger Curvature
We will perform 3 replicas to benchmark the computation of Menger curvature on both Tau protein and tubulin chain A molecular dynamics data. The Menger curvature provides a geometric measure of the curvature through three consecutive points along the protein backbone.


In [18]:
def benchmark_menger(topology_file, trajectory_file, n_replicas=3):
    """
    Benchmark Menger curvature analysis performance.
    
    Parameters:
    -----------
    topology_file : str
        Path to the topology file (PSF/PDB)
    trajectory_file : str
        Path to the trajectory file (DCD)
    n_replicas : int, optional
        Number of benchmark replicas to run (default=3)
        
    Returns:
    --------
    tuple
        (mean_time, std_time) in seconds
    """
    times = []
    
    for _ in range(n_replicas):
        start_time = time.perf_counter()
        
        # Load the system
        u = mda.Universe(topology_file, trajectory_file)
        
        # Calculate Menger curvature
        menger_analyser = MengerCurvature(
            u,
            select="name CA",
            spacing=2
        )
        menger_analyser.run()
        
        end_time = time.perf_counter()
        execution_time = end_time - start_time
        times.append(execution_time)

    mean_time = np.mean(times)
    std_time = np.std(times)
    
    return mean_time, std_time

In [19]:
# Benchmark Menger curvature on Tau protein MD
mean_time, std_time = benchmark_menger(topology_file, trajectory_file, n_replicas=3)

print(f"Mean execution time: {mean_time:.2f} ± {std_time:.2f} seconds")

Mean execution time: 4.54 ± 0.50 seconds


In [27]:
# Benchmark Menger curvature on Tubulin chain A MD
mean_time, std_time = benchmark_menger(files.TUBULIN_CHAIN_A_PDB, files.TUBULIN_CHAIN_A_DCD, n_replicas=3)

print(f"Mean execution time: {mean_time:.2f} ± {std_time:.3f} seconds")

Mean execution time: 0.23 ± 0.003 seconds


In [25]:
def benchmark_menger_parallel(topology_file, trajectory_file, n_replicas=3, n_workers=4):
    """
    Benchmark parallel Menger curvature analysis performance.
    
    Parameters:
    -----------
    topology_file : str
        Path to the topology file (PSF/PDB)
    trajectory_file : str
        Path to the trajectory file (DCD)
    n_replicas : int, optional
        Number of benchmark replicas to run (default=3)
    n_workers : int, optional
        Number of parallel workers to use (default=4)
        
    Returns:
    --------
    tuple
        (mean_time, std_time) in seconds
    """
    times = []
    
    for _ in range(n_replicas):
        start_time = time.perf_counter()
        
        # Load the system
        u = mda.Universe(topology_file, trajectory_file)
        
        # Calculate Menger curvature using parallel processing
        menger_analyser = MengerCurvature(
            u,
            select="name CA",
            spacing=2,
            n_workers=n_workers
        )
        menger_analyser.run_parallel()
        
        end_time = time.perf_counter()
        execution_time = end_time - start_time
        times.append(execution_time)

    mean_time = np.mean(times)
    std_time = np.std(times)
    
    return mean_time, std_time

In [22]:
# Benchmark Menger curvature on Tau protein MD
mean_time, std_time = benchmark_menger_parallel(topology_file, trajectory_file, n_replicas=3)

print(f"Mean execution time: {mean_time:.2f} ± {std_time:.2f} seconds")

Mean execution time: 2.84 ± 0.05 seconds


In [28]:
# Benchmark Menger curvature on Tau protein MD
mean_time, std_time = benchmark_menger_parallel(files.TUBULIN_CHAIN_A_PDB, files.TUBULIN_CHAIN_A_DCD, n_replicas=3)

print(f"Mean execution time: {mean_time:.2f} ± {std_time:.3f} seconds")

Mean execution time: 0.19 ± 0.009 seconds
